# Using FABRIC GPUs

Your compute nodes can include GPUs. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use GPU devices on FABRIC.

## Configure the Environment

In [11]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

## Setup the Experiment

#### Import FABRIC API

In [12]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

In [13]:
print(f"FABRIC_ORCHESTRATOR_HOST: {os.environ['FABRIC_ORCHESTRATOR_HOST']}")
print(f"FABRIC_CREDMGR_HOST:      {os.environ['FABRIC_CREDMGR_HOST']}")
print(f"FABRIC_TOKEN_LOCATION:    {os.environ['FABRIC_TOKEN_LOCATION']}")


slice_manager = SliceManager(oc_host=os.environ['FABRIC_ORCHESTRATOR_HOST'], 
                             cm_host=os.environ['FABRIC_CREDMGR_HOST'] ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

FABRIC_ORCHESTRATOR_HOST: orchestrator.fabric-testbed.net
FABRIC_CREDMGR_HOST:      cm.fabric-testbed.net
FABRIC_TOKEN_LOCATION:    /Users/pruth/work/fabric_token.json


NameError: name 'SliceManager' is not defined

## Create a Node

The cell below creates a slice that contains a single node. The node includes a GPU component.

### Set the Slice Name and FABRIC Site

In [14]:
slice_name="MySliceGPU"
site="MAX"
node_name='Node1'
username='ubuntu'
image = 'default_ubuntu_20'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

gpu_name='gpu1'

In [15]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=cores, ram=ram, disk=disk)
    node.set_image(image)
    
    #Add an NVME Drive
    #node.add_component(model='GPU_TeslaT4', name=gpu_name)
    # OR
    node.add_component(model='GPU_RTX6000', name=gpu_name)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

Waiting for slice ........ Slice state: StableOK


## Get the Slice

Retrieve the node information and save the management IP addresses.

In [19]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slice Error: {e}")

Slice: MySliceGPU


## Get the Node

Retrieve the node information and save the management IP address.


In [20]:
try:
    node = slice.get_node(node_name) 
    print(f"Node Name        : {node.get_name()}")
    print(f"Management IP    : {node.get_management_ip()}")
    print(f"SSH Command      : {node.get_ssh_command()}")
    print()

    gpu1 = node.get_component(gpu_name)
    print(f"GPU Name         : {gpu1.get_name()}")
    print(f"Details          : {gpu1.get_details()}")
    print(f"Units            : {gpu1.get_unit()}")
    print(f"PCI Address      : {gpu1.get_pci_addr()}")
    print(f"Model            : {gpu1.get_model()}")
    print(f"Type             : {gpu1.get_type()}")  
except Exception as e:
    print(f"Error: {e}")



Node Name        : Node1
Management IP    : 63.239.135.84
SSH Command      : ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net ubuntu@63.239.135.84

GPU Name         : gpu1
Details          : NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)
Units            : 1
PCI Address      : 0000:25:00.0
Model            : RTX6000
Type             : GPU


### GPU PCI Device

Run the command <code>lspci</code> to see your GPU PCI device(s). This is the raw GPU PCI device that is not yet configured for use.  You can use the GPUs as you would any GPUs.

An example of using the GPUs is coming soon.

View node1's GPU

In [21]:
command = 'lspci'
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Fail: {e}")

stdout: 00:00.0 Host bridge: Intel Corporation 440FX - 82441FX PMC [Natoma] (rev 02)
00:01.0 ISA bridge: Intel Corporation 82371SB PIIX3 ISA [Natoma/Triton II]
00:01.1 IDE interface: Intel Corporation 82371SB PIIX3 IDE [Natoma/Triton II]
00:01.2 USB controller: Intel Corporation 82371SB PIIX3 USB [Natoma/Triton II] (rev 01)
00:01.3 Bridge: Intel Corporation 82371AB/EB/MB PIIX4 ACPI (rev 03)
00:02.0 VGA compatible controller: Cirrus Logic GD 5446
00:03.0 Ethernet controller: Red Hat, Inc. Virtio network device
00:04.0 SCSI storage controller: Red Hat, Inc. Virtio block device
00:05.0 Unclassified device [00ff]: Red Hat, Inc. Virtio memory balloon
00:06.0 Unclassified device [00ff]: Red Hat, Inc. Virtio RNG
00:07.0 3D controller: NVIDIA Corporation TU102GL [Quadro RTX 6000/8000] (rev a1)

stderr: 


## Cleanup Your Experiment

In [22]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()